In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
raw_documents = WikipediaLoader(query="Atelectasis").load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents([raw_documents[0]]) # should I only take the first? which is the most relevant?

In [3]:
from langchain.indexes import GraphIndexCreator
from langchain.graphs.networkx_graph import KnowledgeTriple
from langchain.llms import OpenAI

In [4]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [5]:
graph = None
for d in documents:
    g_temp = index_creator.from_text(d.page_content)
    if graph is None:
        graph = g_temp
    else:
        triplets = graph.get_triples()
        for t in triplets:
            graph.add_triple(knowledge_triple=KnowledgeTriple(*t))

In [6]:
from langchain.chains import GraphQAChain

In [7]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [8]:
chain.run("What is Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Atelectasis is the collapse or closure of a lung, where the alveoli are deflated down to little or no volume. It is often called a collapsed lung and is not synonymous with a pneumothorax. It may occur as a post-operative complication or as a result of surfactant deficiency and can lead to infant respiratory distress syndrome. It is caused by normal exhalation and various medical conditions and is associated with post-op fever.'

In [9]:
chain.run("What are the symptoms of Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Symptoms of Atelectasis include shortness of breath, chest pain, coughing, and difficulty breathing.'

In [10]:
chain.run("What can cause Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Atelectasis can be caused by normal exhalation, various medical conditions, and surfactant deficiency. It can also occur as a post-operative complication or as a result of infant respiratory distress syndrome.'

In [11]:
chain.run("What clinical features radiologists need for diagnosing Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Radiologists need to look for signs of unilateral collapse or closure of a lung, deflated alveoli, and post-operative fever. They may also need to consider other medical conditions that could be causing the Atelectasis.'

In [12]:
chain.run("what are radiographic features of Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Radiographic features of Atelectasis include a decrease in lung volume, a decrease in the size of the affected lung, and a decrease in the density of the affected lung.'